In [12]:
import os
from datetime import datetime
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import pandas as pd
import plotly.express as px

app = dash.Dash(__name__)

# Directory containing CSV files
directory = "C:/Users/peter/Desktop/Speciale/DS_thesis/P10. 2024. Pilot dashboard (1)"
output_directory = "C:/Users/peter/Desktop/Speciale/DS_thesis/P10, 2024, saved files"

# List of variable names to filter
variable_names = [
    "AI Values_78TT001 - Analog input",
    "AI Values_78TT002 - Analog input",
    "AI Values_10TT002 - Analog input",
    "AI Values_20TTC001 - Analog input",
    "AI Values_20FTC003 - analog input",
    "AI Values_78FT001 - Analog input",
    "AI Values_20FTC002 - Analog input",
    "AI Values_20XTC001 - Analog input",
    "AI Values_20XTC002 - Analog input",
    "AI Values_20XT004 - Analog input",
    "AI Values_20XTC003 - Analog input",
    "AI Values_10PT001 - Analog input",
    "30P001.HMI.DATA_2",
    "30P002.HMI.DATA_2",
    "30P001.HMI.STATUS",
    "AO Values_10R001",
    "AO Values_10R002",
    "AO Values_10R003",
    "AI Values_20PT004 - Analog input",
    "AI Values_78PT002 - Analog input",
    "AI Values_78PT001 - Analog input",
]

# Dictionary to store merged DataFrames for all variables
merged_dataframes = {var: pd.DataFrame() for var in variable_names}

# Store filename prefixes entered by the user
filename_prefixes = []

# List CSV files in the directory
def list_csv_files(directory):
    return [f for f in os.listdir(directory) if f.endswith('.csv')]

# Filter files by date
def filter_files_by_date(files, start_date, end_date):
    filtered_files = []
    for file in files:
        try:
            # Extract date from filename (assuming format: Data_log0_YYYYMMDD_HHMMSS.csv)
            file_date_str = file.split('_')[2]
            file_date = datetime.strptime(file_date_str, "%Y%m%d")
            if start_date <= file_date <= end_date:
                filtered_files.append(file)
        except Exception as e:
            print(f"Error processing file {file}: {e}")
    return filtered_files

# Function to process a single CSV file and extract variables into DataFrames
def process_csv_file(directory, csv_file, variable_names):
    # Change to the correct working directory
    os.chdir(directory)
    
    # Read the CSV file (assuming ';' as delimiter)
    df = pd.read_csv(csv_file, delimiter=';', on_bad_lines='skip')
    
    # Dictionary to store DataFrames for each variable
    dataframes = {}
    
    # Iterate over each variable name and filter the rows
    for var_name in variable_names:
        filtered_df = df[df['VarName'] == var_name]
        if not filtered_df.empty:
            dataframes[var_name] = filtered_df
            print(f"Extracted {var_name} with {len(filtered_df)} rows.")
    
    return dataframes

# Function to extract filename prefixes from the saved files in the output directory
def extract_prefixes_from_saved_files(output_directory):
    prefixes = set()
    for file in os.listdir(output_directory):
        if file.endswith('.csv'):
            # Extract prefix by splitting the filename (assumes format: prefix_variable.csv)
            prefix = file.split('_')[0]
            prefixes.add(prefix)
    return sorted(list(prefixes))

# Dash layout
app.layout = html.Div([
    # Centered dropdowns at the top
    html.Div([
        dcc.Dropdown(
            id='prefix-dropdown',
            placeholder='Select saved file prefix',
            style={'width': '50%', 'marginBottom': '10px'},
            multi=True  # Enable multi-selection
        ),
        dcc.Dropdown(
            id='variable-dropdown',
            placeholder='Select variable',
            style={'width': '50%', 'marginBottom': '10px'},
            multi=True  # Enable multi-selection for variables
        ),
        # Add RadioItems for switching between Elapsed Time and Absolute Time
        dcc.RadioItems(
            id='time-mode-switch',
            options=[
                {'label': 'Elapsed Time', 'value': 'elapsed'},
                {'label': 'Absolute Time', 'value': 'absolute'},
            ],
            value='elapsed',  # Default to 'Elapsed Time'
            labelStyle={'display': 'inline-block', 'margin': '0 10px'},
            style={'marginTop': '10px', 'marginBottom': '10px'}
        )
    ], style={
        'display': 'flex',
        'flexDirection': 'column',
        'alignItems': 'center',
        'justifyContent': 'center',
        'width': '100%',
        'marginBottom': '20px'
    }),
    
    # Main content layout: Define Batch section and file list
    html.Div([
        # Define Batch (Date Picker and Filename Input) on the left
        html.Div([
            html.H3('Define Batch', style={'marginBottom': '10px'}),
            dcc.DatePickerRange(
                id='date-picker-range',
                start_date=datetime(2024, 1, 1),
                end_date=datetime(2024, 12, 31),
                display_format='YYYY-MM-DD',
                style={'marginBottom': '10px', 'width': '100%'}
            ),
            dcc.Input(
                id='filename-input',
                type='text',
                placeholder='Enter filename prefix',
                style={'display': 'block', 'margin': '10px 0', 'width': '19%'}
            ),
            html.Button('Confirm', id='confirm-button', n_clicks=0, style={'display': 'block', 'margin': '10px 0'}),
            # File list should appear here, underneath the confirm button
            html.Div(id='file-list', style={'whiteSpace': 'pre-line', 'marginTop': '10px'}),
            html.Div(id='save-status', style={'marginTop': '10px'})
        ], style={
            'textAlign': 'left',
            'width': '30%',
            'padding': '20px',
            'flex': '1'
        }),
        
        # The graph to visualize data on the right
        html.Div([
            dcc.Graph(id='variable-graph')
        ], style={
            'textAlign': 'center',
            'width': '70%',
            'padding': '20px',
            'flex': '2'
        })
        
    ], style={'display': 'flex', 'width': '100%', 'marginTop': '30px'})
], style={'width': '100%', 'padding': '20px'})

# Callback to handle file selection, processing, and saving
@app.callback(
    Output('file-list', 'children'),
    Output('save-status', 'children'),
    Output('prefix-dropdown', 'options'),
    [Input('confirm-button', 'n_clicks')],
    [State('date-picker-range', 'start_date'),
     State('date-picker-range', 'end_date'),
     State('filename-input', 'value')]
)
def update_file_list(n_clicks, start_date, end_date, filename_prefix):
    if n_clicks > 0 and start_date and end_date and filename_prefix:
        # Convert dates from string to datetime
        start_date = datetime.fromisoformat(start_date)
        end_date = datetime.fromisoformat(end_date)
        
        # List available CSV files
        csv_files = list_csv_files(directory)
        
        # Filter files by the chosen date range
        selected_files = filter_files_by_date(csv_files, start_date, end_date)
        
        if not selected_files:
            return "No files selected.", "", []

        # Create output directory if it doesn't exist
        os.makedirs(output_directory, exist_ok=True)

        global merged_dataframes  # Access the global dictionary to store merged DataFrames
        
        # Reset the merged_dataframes for new processing
        merged_dataframes = {var: pd.DataFrame() for var in variable_names}
        
        # Process each selected file and merge data for each variable
        for file in selected_files:
            csv_file_path = os.path.join(directory, file)
            
            # Extract the variable-specific DataFrames for this file
            dataframes = process_csv_file(directory, csv_file_path, variable_names)
            
            # Merge the data for each variable
            for var_name, df in dataframes.items():
                if not df.empty:
                    merged_dataframes[var_name] = pd.concat([merged_dataframes[var_name], df], ignore_index=True)
        
        # Save the merged DataFrames into CSV files (one per variable)
        saved_files = []
        for var_name, df in merged_dataframes.items():
            if not df.empty:
                output_file = os.path.join(output_directory, f"{filename_prefix}_{var_name}.csv")
                df.to_csv(output_file, index=False)
                saved_files.append(output_file)
                print(f"Saved {output_file}")
        
        # Track the used filename prefix for the dropdown
        if filename_prefix not in filename_prefixes:
            filename_prefixes.append(filename_prefix)
        
        # Include prefixes from the saved files in the output directory
        all_prefixes = sorted(list(set(filename_prefixes + extract_prefixes_from_saved_files(output_directory))))
        
        # Update the options for the prefix dropdown
        prefix_options = [{'label': prefix, 'value': prefix} for prefix in all_prefixes]
        
        return "\n".join(selected_files), f"Files processed and saved with prefix: {filename_prefix}. Files saved: {len(saved_files)}", prefix_options
    
    return "No files selected.", "", [{'label': prefix, 'value': prefix} for prefix in extract_prefixes_from_saved_files(output_directory)]

# Callback to update the variable dropdown based on selected prefixes
@app.callback(
    Output('variable-dropdown', 'options'),
    [Input('prefix-dropdown', 'value')]
)
def update_variable_dropdown(selected_prefixes):
    if selected_prefixes:
        # Assuming multiple prefixes are selected, iterate through them
        variable_options = []
        for prefix in selected_prefixes:
            # For each prefix, list the variables, appending the prefix to distinguish them
            for var_name in variable_names:
                labeled_var_name = f"{prefix}_{var_name}"
                variable_options.append({'label': labeled_var_name, 'value': labeled_var_name})
        return variable_options
    return []

# Callback to update the graph based on selected variables and time mode
@app.callback(
    Output('variable-graph', 'figure'),
    [Input('variable-dropdown', 'value'),
     Input('time-mode-switch', 'value')]  # Add time mode switch as input
)
def update_graph(selected_variables, time_mode):
    if selected_variables:
        # Prepare a DataFrame to hold all selected variable data
        all_data = pd.DataFrame()

        for var in selected_variables:
            prefix, var_name = var.split('_', 1)
            file_path = os.path.join(output_directory, f"{prefix}_{var_name}.csv")
            
            # Load the CSV file corresponding to the selected variable
            if os.path.exists(file_path):
                df = pd.read_csv(file_path)
                
                # Ensure the DataFrame has needed columns for plotting
                if 'TimeString' in df.columns and 'VarValue' in df.columns:
                    # Convert TimeString to datetime with the correct format (day-month-year)
                    df['TimeString'] = pd.to_datetime(df['TimeString'], format='%d-%m-%Y %H:%M:%S', errors='coerce')
                    
                    # Calculate elapsed time in seconds
                    df['ElapsedTime'] = (df['TimeString'] - df['TimeString'].min()).dt.total_seconds()
                    
                    df['Variable'] = var_name  # Add a column to distinguish the variable
                    all_data = pd.concat([all_data, df], ignore_index=True)
        
        # Check if we have data to plot
        if not all_data.empty:
            # Choose between 'ElapsedTime' or 'TimeString' based on the time_mode
            if time_mode == 'elapsed':
                x_axis = 'ElapsedTime'
                x_label = 'Elapsed Time (seconds)'
            else:
                x_axis = 'TimeString'
                x_label = 'Time'
            
            # Plot using the selected x-axis
            fig = px.line(
                all_data, 
                x=x_axis, 
                y='VarValue', 
                color='Variable',
                labels={x_axis: x_label},
                title="Selected Variable Data Over Time"
            )
            return fig
    
    # Return an empty figure if no variables are selected
    return px.line()

# Run the Dash server
if __name__ == '__main__':
    # Initialize the dropdown with any already saved prefixes in the output directory
    filename_prefixes = extract_prefixes_from_saved_files(output_directory)
    
    app.run_server(debug=True, port=8091)